In [1]:
from dataclasses import dataclass
from io import BytesIO, StringIO
from time import sleep
from typing import Dict, List, Tuple, Union
from random import choices, randint, uniform

import pygame
import pygame.mixer

from midiutil import MIDIFile

pygame 2.1.2 (SDL 2.0.18, Python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [17]:
NOTE_LENGTH = [1 / 32, 1 / 16, 1 / 8, 1 / 4]  # w.r.t. bar
NOTE_PROB = [0.0, 0.8, 0.2, 0.0]

SILENCE_PROB = 0.25

MELODY_PATTERNS = ["AABA", "ABAB", "ABAC", "ABCB", "AAAB"]

BEAT_PER_BAR = 4
VOLUME = 100

In [4]:
@dataclass
class MidiData:
    note: int
    start_time: float
    duration: float

major_scale = [60, 62, 64, 65, 67, 69, 71, 72] # MIDI note number
minor_scale = [60, 62, 63, 65, 67, 68, 70, 72] # MIDI note number

def generate_midi(scale: str='C', major: bool=True, bpm: int=126) -> BytesIO:
    melody_pattern = MELODY_PATTERNS[randint(0, len(MELODY_PATTERNS) - 1)]
    
    pattern_memo: Dict[str, List[MidiData]] = {}
    for pattern in melody_pattern:
        if pattern in pattern_memo:
            continue
        time = 0
        pattern_memo[pattern] = []
        while time < BEAT_PER_BAR:
            is_silence = uniform(0, 1) <= SILENCE_PROB
            if is_silence:
                duration = choices(NOTE_LENGTH, NOTE_PROB)[0] * BEAT_PER_BAR
                time += duration
                continue
            note_id = randint(0, len(major_scale) - 1)
            note = major_scale[note_id] if major else minor_scale[note_id]
            duration = choices(NOTE_LENGTH, NOTE_PROB)[0] * BEAT_PER_BAR
            if time + duration > BEAT_PER_BAR:
                duration = BEAT_PER_BAR - time
            pattern_memo[pattern].append(MidiData(note=note, start_time=time, duration=duration))
            time += duration
    
    midi = MIDIFile(1)
    midi.addTempo(0, 0, bpm)
    bar_num = 0
    for pattern in melody_pattern:
        for midi_data in pattern_memo[pattern]:
            time = midi_data.start_time + bar_num * BEAT_PER_BAR
            midi.addNote(0, 0, midi_data.note, time, midi_data.duration, VOLUME)
        bar_num += 1
    
    mem_file = BytesIO()
    midi.writeFile(mem_file)
    return mem_file
#     with open("minor-scale.mid", "wb") as output_file:
#         midi.writeFile(output_file)

In [23]:
mem_file = generate_midi(major=False, bpm=130)
pygame.init()
pygame.mixer.init()
mem_file.seek(0)  # THIS IS CRITICAL, OTHERWISE YOU GET THAT ERROR!
pygame.mixer.music.load(mem_file)

pygame.mixer.music.play()
while pygame.mixer.music.get_busy():
    sleep(1)
print("Done!")

Done!
